In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# !git pull

In [3]:
import tensorflow as tf
from glob import glob
import malaya_speech.config
import malaya_speech.train.model.mini_jasper as mini_jasper
import malaya_speech.train.model.ctc as ctc
import malaya_speech.train as train
import numpy as np


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.tf_featurization.STTFeaturizer(config)
n_mels = config['num_feature_bins']
initial_learning_rate = 1e-3
max_gradient_norm = 5.0
train_steps = 1000

In [5]:
def preprocess_inputs(example):
    s = featurizer.vectorize(example['waveforms'])
    mel_fbanks = tf.reshape(s, (-1, n_mels))
    length = tf.cast(tf.shape(mel_fbanks)[0], tf.int32)
    length = tf.expand_dims(length, 0)
    example['inputs'] = mel_fbanks
    example['inputs_length'] = length

    return example

In [6]:
def parse(serialized_example):

    data_fields = {
        'waveforms': tf.VarLenFeature(tf.float32),
        'targets': tf.VarLenFeature(tf.int64),
    }
    features = tf.parse_single_example(
        serialized_example, features = data_fields
    )
    for k in features.keys():
        features[k] = features[k].values

    features = preprocess_inputs(features)

    keys = list(features.keys())
    for k in keys:
        if k not in ['inputs', 'inputs_length', 'targets']:
            features.pop(k, None)

    return features


def get_dataset(path, batch_size = 32, shuffle_size = 32, thread_count = 24):
    def get():
        files = glob(path)
        dataset = tf.data.TFRecordDataset(files)
        dataset = dataset.shuffle(shuffle_size)
        dataset = dataset.map(parse, num_parallel_calls = thread_count)
        dataset = dataset.padded_batch(
            batch_size,
            padded_shapes = {
                'inputs': tf.TensorShape([None, n_mels]),
                'inputs_length': tf.TensorShape([None]),
                'targets': tf.TensorShape([None]),
            },
            padding_values = {
                'inputs': tf.constant(0, dtype = tf.float32),
                'inputs_length': tf.constant(0, dtype = tf.int32),
                'targets': tf.constant(0, dtype = tf.int64),
            },
        )
        dataset = dataset.repeat()
        return dataset

    return get

In [7]:
def model_fn(features, labels, mode, params):

    model = mini_jasper.Model(
        features['inputs'], features['inputs_length'][:, 0], mode = 'train'
    )
    logits = tf.layers.dense(
        model.logits['outputs'], malaya_speech.char.VOCAB_SIZE
    )
    seq_lens = model.logits['src_length']

    targets_int32 = tf.cast(features['targets'], tf.int32)

    mean_error, sum_error, sum_weight = ctc.loss.ctc_loss(
        logits, targets_int32, seq_lens
    )

    loss = mean_error
    accuracy = ctc.metrics.ctc_sequence_accuracy(
        logits, targets_int32, seq_lens
    )

    tf.identity(loss, 'train_loss')
    tf.identity(accuracy, name = 'train_accuracy')

    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_or_create_global_step()
        learning_rate = tf.train.polynomial_decay(
            initial_learning_rate,
            global_step,
            train_steps,
            end_learning_rate = 1e-4,
            power = 1.0,
            cycle = False,
        )
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
        train_op = optimizer.minimize(loss, global_step = global_step)
        estimator_spec = tf.estimator.EstimatorSpec(
            mode = mode, loss = loss, train_op = train_op
        )

    elif mode == tf.estimator.ModeKeys.EVAL:

        estimator_spec = tf.estimator.EstimatorSpec(
            mode = tf.estimator.ModeKeys.EVAL,
            loss = loss,
            eval_metric_ops = {
                'accuracy': ctc.metrics.ctc_sequence_accuracy_estimator(
                    logits, targets_int32, seq_lens
                ),
                'WER': ctc.metrics.word_error_rate_estimator(
                    logits, targets_int32
                ),
            },
        )

    return estimator_spec

In [8]:
train_hooks = [
    tf.train.LoggingTensorHook(
        ['train_accuracy', 'train_loss'], every_n_iter = 1
    )
]
train_dataset = get_dataset('tolong-sebut/data/tolong-sebut-train*')
dev_dataset = get_dataset('tolong-sebut/data/tolong-sebut-dev*')

In [9]:
!rm -rf asr-minijasper

In [10]:
train.run_training(
    train_fn = train_dataset,
    model_fn = model_fn,
    model_dir = 'asr-minijasper',
    num_gpus = 1,
    log_step = 1,
    save_checkpoint_step = 200,
    max_steps = train_steps,
    eval_fn = dev_dataset,
    train_hooks = train_hooks,
)



INFO:tensorflow:Using config: {'_model_dir': 'asr-minijasper', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 200, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f44d1b5a4a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluatio

INFO:tensorflow:train_accuracy = 0.0, train_loss = 57.889862 (0.495 sec)
INFO:tensorflow:loss = 57.889862, step = 15 (0.495 sec)
INFO:tensorflow:global_step/sec: 2.21574
INFO:tensorflow:train_accuracy = 0.0039011703, train_loss = 57.588512 (0.451 sec)
INFO:tensorflow:loss = 57.588512, step = 16 (0.451 sec)
INFO:tensorflow:global_step/sec: 2.4793
INFO:tensorflow:train_accuracy = 0.0050188205, train_loss = 60.14511 (0.403 sec)
INFO:tensorflow:loss = 60.14511, step = 17 (0.403 sec)
INFO:tensorflow:global_step/sec: 2.11518
INFO:tensorflow:train_accuracy = 0.0027137042, train_loss = 49.564636 (0.473 sec)
INFO:tensorflow:loss = 49.564636, step = 18 (0.472 sec)
INFO:tensorflow:global_step/sec: 2.54407
INFO:tensorflow:train_accuracy = 0.0013262599, train_loss = 58.36293 (0.393 sec)
INFO:tensorflow:loss = 58.36293, step = 19 (0.394 sec)
INFO:tensorflow:global_step/sec: 2.09752
INFO:tensorflow:train_accuracy = 0.0, train_loss = 64.0806 (0.477 sec)
INFO:tensorflow:loss = 64.0806, step = 20 (0.476

INFO:tensorflow:train_accuracy = 0.040920716, train_loss = 36.700653 (0.476 sec)
INFO:tensorflow:loss = 36.700653, step = 62 (0.475 sec)
INFO:tensorflow:global_step/sec: 2.35393
INFO:tensorflow:train_accuracy = 0.040897097, train_loss = 31.582521 (0.425 sec)
INFO:tensorflow:loss = 31.582521, step = 63 (0.426 sec)
INFO:tensorflow:global_step/sec: 2.05959
INFO:tensorflow:train_accuracy = 0.043137256, train_loss = 34.118744 (0.485 sec)
INFO:tensorflow:loss = 34.118744, step = 64 (0.485 sec)
INFO:tensorflow:global_step/sec: 1.94399
INFO:tensorflow:train_accuracy = 0.062834226, train_loss = 49.79178 (0.515 sec)
INFO:tensorflow:loss = 49.79178, step = 65 (0.515 sec)
INFO:tensorflow:global_step/sec: 2.15142
INFO:tensorflow:train_accuracy = 0.017971758, train_loss = 34.08287 (0.465 sec)
INFO:tensorflow:loss = 34.08287, step = 66 (0.465 sec)
INFO:tensorflow:global_step/sec: 2.07075
INFO:tensorflow:train_accuracy = 0.016169155, train_loss = 35.98874 (0.483 sec)
INFO:tensorflow:loss = 35.98874, s

INFO:tensorflow:loss = 29.263489, step = 108 (0.500 sec)
INFO:tensorflow:global_step/sec: 2.01096
INFO:tensorflow:train_accuracy = 0.3606341, train_loss = 27.04092 (0.497 sec)
INFO:tensorflow:loss = 27.04092, step = 109 (0.497 sec)
INFO:tensorflow:global_step/sec: 1.87763
INFO:tensorflow:train_accuracy = 0.449935, train_loss = 27.962086 (0.533 sec)
INFO:tensorflow:loss = 27.962086, step = 110 (0.533 sec)
INFO:tensorflow:global_step/sec: 1.93189
INFO:tensorflow:train_accuracy = 0.49132177, train_loss = 27.491446 (0.518 sec)
INFO:tensorflow:loss = 27.491446, step = 111 (0.518 sec)
INFO:tensorflow:global_step/sec: 2.14362
INFO:tensorflow:train_accuracy = 0.43960148, train_loss = 31.080612 (0.466 sec)
INFO:tensorflow:loss = 31.080612, step = 112 (0.466 sec)
INFO:tensorflow:global_step/sec: 2.18577
INFO:tensorflow:train_accuracy = 0.5111402, train_loss = 25.162666 (0.458 sec)
INFO:tensorflow:loss = 25.162666, step = 113 (0.458 sec)
INFO:tensorflow:global_step/sec: 2.02662
INFO:tensorflow:tr

INFO:tensorflow:global_step/sec: 1.84593
INFO:tensorflow:train_accuracy = 0.5012854, train_loss = 25.363762 (0.542 sec)
INFO:tensorflow:loss = 25.363762, step = 155 (0.542 sec)
INFO:tensorflow:global_step/sec: 1.77019
INFO:tensorflow:train_accuracy = 0.5313352, train_loss = 21.51256 (0.565 sec)
INFO:tensorflow:loss = 21.51256, step = 156 (0.564 sec)
INFO:tensorflow:global_step/sec: 2.22995
INFO:tensorflow:train_accuracy = 0.48995984, train_loss = 25.39513 (0.448 sec)
INFO:tensorflow:loss = 25.39513, step = 157 (0.449 sec)
INFO:tensorflow:global_step/sec: 2.04456
INFO:tensorflow:train_accuracy = 0.49210206, train_loss = 40.815117 (0.489 sec)
INFO:tensorflow:loss = 40.815117, step = 158 (0.489 sec)
INFO:tensorflow:global_step/sec: 1.78673
INFO:tensorflow:train_accuracy = 0.529563, train_loss = 27.555763 (0.560 sec)
INFO:tensorflow:loss = 27.555763, step = 159 (0.560 sec)
INFO:tensorflow:global_step/sec: 1.93738
INFO:tensorflow:train_accuracy = 0.50530505, train_loss = 23.255363 (0.516 se

...

INFO:tensorflow:loss = 14.665762, step = 791 (0.486 sec)
INFO:tensorflow:global_step/sec: 1.90767
INFO:tensorflow:train_accuracy = 0.56993467, train_loss = 14.51587 (0.524 sec)
INFO:tensorflow:loss = 14.51587, step = 792 (0.525 sec)
INFO:tensorflow:global_step/sec: 2.34502
INFO:tensorflow:train_accuracy = 0.5693527, train_loss = 12.016697 (0.426 sec)
INFO:tensorflow:loss = 12.016697, step = 793 (0.426 sec)
INFO:tensorflow:global_step/sec: 2.05305
INFO:tensorflow:train_accuracy = 0.5752688, train_loss = 13.68531 (0.487 sec)
INFO:tensorflow:loss = 13.68531, step = 794 (0.487 sec)
INFO:tensorflow:global_step/sec: 1.96775
INFO:tensorflow:train_accuracy = 0.5470738, train_loss = 15.799996 (0.508 sec)
INFO:tensorflow:loss = 15.799996, step = 795 (0.508 sec)
INFO:tensorflow:global_step/sec: 1.83103
INFO:tensorflow:train_accuracy = 0.5737052, train_loss = 12.753255 (0.546 sec)
INFO:tensorflow:loss = 12.753255, step = 796 (0.546 sec)
INFO:tensorflow:global_step/sec: 1.98852
INFO:tensorflow:trai

INFO:tensorflow:train_accuracy = 0.5643045, train_loss = 12.0097 (0.476 sec)
INFO:tensorflow:loss = 12.0097, step = 837 (0.476 sec)
INFO:tensorflow:global_step/sec: 2.04721
INFO:tensorflow:train_accuracy = 0.5883152, train_loss = 11.255579 (0.488 sec)
INFO:tensorflow:loss = 11.255579, step = 838 (0.488 sec)
INFO:tensorflow:global_step/sec: 2.00841
INFO:tensorflow:train_accuracy = 0.5595855, train_loss = 14.105899 (0.498 sec)
INFO:tensorflow:loss = 14.105899, step = 839 (0.498 sec)
INFO:tensorflow:global_step/sec: 1.98664
INFO:tensorflow:train_accuracy = 0.57014155, train_loss = 13.172909 (0.503 sec)
INFO:tensorflow:loss = 13.172909, step = 840 (0.504 sec)
INFO:tensorflow:global_step/sec: 1.96491
INFO:tensorflow:train_accuracy = 0.5606258, train_loss = 13.402395 (0.509 sec)
INFO:tensorflow:loss = 13.402395, step = 841 (0.509 sec)
INFO:tensorflow:global_step/sec: 2.16097
INFO:tensorflow:train_accuracy = 0.575, train_loss = 11.21554 (0.463 sec)
INFO:tensorflow:loss = 11.21554, step = 842 

INFO:tensorflow:loss = 10.816273, step = 883 (0.482 sec)
INFO:tensorflow:global_step/sec: 1.92051
INFO:tensorflow:train_accuracy = 0.59495354, train_loss = 11.062202 (0.520 sec)
INFO:tensorflow:loss = 11.062202, step = 884 (0.520 sec)
INFO:tensorflow:global_step/sec: 2.05733
INFO:tensorflow:train_accuracy = 0.56031126, train_loss = 11.085577 (0.486 sec)
INFO:tensorflow:loss = 11.085577, step = 885 (0.486 sec)
INFO:tensorflow:global_step/sec: 2.18522
INFO:tensorflow:train_accuracy = 0.55927837, train_loss = 14.528601 (0.458 sec)
INFO:tensorflow:loss = 14.528601, step = 886 (0.457 sec)
INFO:tensorflow:global_step/sec: 2.0263
INFO:tensorflow:train_accuracy = 0.57588077, train_loss = 10.444466 (0.494 sec)
INFO:tensorflow:loss = 10.444466, step = 887 (0.494 sec)
INFO:tensorflow:global_step/sec: 2.11165
INFO:tensorflow:train_accuracy = 0.5770235, train_loss = 11.039472 (0.473 sec)
INFO:tensorflow:loss = 11.039472, step = 888 (0.472 sec)
INFO:tensorflow:global_step/sec: 1.8229
INFO:tensorflow

INFO:tensorflow:train_accuracy = 0.58539766, train_loss = 10.408207 (0.498 sec)
INFO:tensorflow:loss = 10.408207, step = 930 (0.497 sec)
INFO:tensorflow:global_step/sec: 1.9597
INFO:tensorflow:train_accuracy = 0.5983828, train_loss = 9.776345 (0.510 sec)
INFO:tensorflow:loss = 9.776345, step = 931 (0.510 sec)
INFO:tensorflow:global_step/sec: 2.01038
INFO:tensorflow:train_accuracy = 0.5793651, train_loss = 11.331739 (0.497 sec)
INFO:tensorflow:loss = 11.331739, step = 932 (0.497 sec)
INFO:tensorflow:global_step/sec: 1.89622
INFO:tensorflow:train_accuracy = 0.58344287, train_loss = 10.3788 (0.527 sec)
INFO:tensorflow:loss = 10.3788, step = 933 (0.527 sec)
INFO:tensorflow:global_step/sec: 2.15092
INFO:tensorflow:train_accuracy = 0.5704698, train_loss = 9.8031 (0.465 sec)
INFO:tensorflow:loss = 9.8031, step = 934 (0.466 sec)
INFO:tensorflow:global_step/sec: 2.00703
INFO:tensorflow:train_accuracy = 0.57215834, train_loss = 10.126661 (0.498 sec)
INFO:tensorflow:loss = 10.126661, step = 935 (

INFO:tensorflow:global_step/sec: 2.17254
INFO:tensorflow:train_accuracy = 0.5868421, train_loss = 8.395511 (0.460 sec)
INFO:tensorflow:loss = 8.395511, step = 977 (0.460 sec)
INFO:tensorflow:global_step/sec: 2.08634
INFO:tensorflow:train_accuracy = 0.5719745, train_loss = 10.685549 (0.479 sec)
INFO:tensorflow:loss = 10.685549, step = 978 (0.479 sec)
INFO:tensorflow:global_step/sec: 2.04638
INFO:tensorflow:train_accuracy = 0.5970549, train_loss = 9.8347 (0.489 sec)
INFO:tensorflow:loss = 9.8347, step = 979 (0.489 sec)
INFO:tensorflow:global_step/sec: 1.98731
INFO:tensorflow:train_accuracy = 0.5978836, train_loss = 9.349877 (0.503 sec)
INFO:tensorflow:loss = 9.349877, step = 980 (0.503 sec)
INFO:tensorflow:global_step/sec: 1.88134
INFO:tensorflow:train_accuracy = 0.5839793, train_loss = 9.844684 (0.532 sec)
INFO:tensorflow:loss = 9.844684, step = 981 (0.532 sec)
INFO:tensorflow:global_step/sec: 2.00263
INFO:tensorflow:train_accuracy = 0.5710526, train_loss = 7.979488 (0.499 sec)
INFO:ten